# Time series classification with Tensorflow & <font color=red>Keras</font> using <font color= #70e514>CNN / ConvNets</font>

## $HAR$ : Humman Activity Recognition

## Example form:

## Time series classification with Tensorflow burakhimmetoglu - August 22, 2017
(https://burakhimmetoglu.com/2017/08/22/time-series-classification-with-tensorflow/)

![](https://burakhimmetoglu.files.wordpress.com/2017/08/ekg-158177_640.png)


# * [MSTC](http://mstc.ssr.upm.es/big-data-track) and MUIT: <font size=5 color='green'>Deep Learning with Tensorflow & Keras</font>

## See Notebooks in GitHub [healthDataScience](https://github.com/healthDataScience)  : $deep-learning-HAR$


![healthDataScience](https://avatars2.githubusercontent.com/u/30838413?s=200&v=4)

In [0]:
! pip install --upgrade tqdm

try:
    from tqdm import tqdm, tqdm_notebook
except ImportError:
    def tqdm(x, *args, **kwargs):
        return x

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
# Imports
import numpy as np
import os

import math


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline



## Some UTILITY functions

In [0]:

import pandas as pd 
import numpy as np
import os


def one_hot(labels, n_class = 6):
	""" One-hot encoding """
	expansion = np.eye(n_class)
	y = expansion[:, labels-1].T
	assert y.shape[1] == n_class, "Wrong number of labels!"

	return y
	


# Human Activity Recognition (HAR) Dataset
## From: [UCI Repository](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)



## Reading data already prepared from orignal UCI Dataset from a Shared Google Drive npz file

In [0]:
! pip install googledrivedownloader

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1VLgB0CK8DjMJ4gstLRC0gA2pVGXy-RYa',
                                    dest_path='./UCI_HAR_Dataset_Drive.npz',
                                    unzip=False)

In [0]:
npzfile = np.load('UCI_HAR_Dataset_Drive.npz')
npzfile.files

X_train=npzfile['arr_0']
X_test=npzfile['arr_1']
labels_train=npzfile['arr_2']
labels_test=npzfile['arr_3']

print('Training data size: ',X_train.shape)
print('Test data size: ',X_test.shape)

print('Training labels size: ',labels_train.shape)
print('Test labels size: ',labels_test.shape)

## Six categories (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING).

[LSTM-Human-Activity-Recognition](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition)

    The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used.

## ... see some labels...

WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING

In [0]:
plt.plot(labels_train[50:250])

## ...see some signals...

In [0]:
plt.subplots(nrows = 3, figsize = (15, 10), sharex = True)
plt.subplot(3,1,1)
plt.plot(X_train[50,:,0])
plt.plot(X_train[50,:,1])
plt.plot(X_train[50,:,2])

plt.subplot(3,1,2)
plt.plot(X_train[80,:,3])
plt.plot(X_train[80,:,4])
plt.plot(X_train[80,:,5])

plt.subplot(3,1,3)
plt.plot(X_train[110,:,0])
plt.plot(X_train[110,:,1])
plt.plot(X_train[110,:,2])

## One-hot-encoding of Labels

In [0]:
y_train = one_hot(labels_train)
y_test = one_hot(labels_test)

## Re-Shape for 2D Conv (1 D??)

In [0]:
X_train=X_train.reshape([X_train.shape[0],X_train.shape[1],X_train.shape[2],1])
X_test=X_test.reshape([X_test.shape[0],X_test.shape[1],X_test.shape[2],1])

print('Train shape=', X_train.shape , '\nTest shape=', X_test.shape)


## 1.   **Define your ConNet model**:


In [0]:

from keras.models import Sequential

from keras.layers import MaxPooling2D, Dropout, Dense, Flatten

from keras.layers import Convolution2D as Conv2D

model = Sequential()
# input: 128x9 images with 1 channel -> (128, 9) tensors.
# this applies 18 convolution filters of size 2x2 each.
model.add(Conv2D(18, (2, 2), activation='relu', input_shape=(128, 9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

## 2.   **Compile your model**. Specify loss functions and optimizers

In [0]:
from keras.optimizers import SGD


model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [0]:
model.summary()

# 3.   **Fit your model**. Train the model using data.

In [0]:
# validation_data allows to see evaluation on test while training
history=model.fit(X_train, y_train,
         epochs=100,
         batch_size=128,
         validation_data=(X_test, y_test),
         verbose=1)

## 4.   **Make predictions**. Use the model to generate predictions on new data.

In [0]:
# Check accuracy on train set

loss_train, accuracy_train = model.evaluate(X_train, y_train, batch_size=128)


print('\nTraining Accuracy=', accuracy_train)

# Check accuracy on test set

loss, accuracy = model.evaluate(X_test, y_test, batch_size=128)


print('\nTest Accuracy=', accuracy)

In [0]:
print('Train Accuracy: ',np.round(history.history['acc'][-1],2))
print('Test_Accuracy: ',np.round(history.history['val_acc'][-1],2))

# Plot the accuracy curves
plt.plot(history.history['acc'],'bo')
plt.plot(history.history['val_acc'],'rX')

# Get predictions / probabilities ...

In [0]:
pred_probabilities= model.predict(X_test)

print('First Five Probs.:\n',pred_probabilities[0:5])

print('\n\nFirst Five Classes:\n',y_test[0:5])

## Confusion Matrix

In [0]:
!pip install pandas_ml

In [0]:
from pandas_ml import ConfusionMatrix

ConfMatrix=ConfusionMatrix(np.argmax(y_test,1), np.argmax(pred_probabilities,1))

ConfMatrix.plot(normalized=True,backend='seaborn')

In [0]:
ConfMatrix.print_stats()